In [95]:
import numpy as np
import pandas as pd

In [96]:
SG = 0.8
Pb = 14.7 #psi
Tb = 60+460 #R

Ts = 90+460 #R
Ps = 250 #psi
Pd = 600 #psi

q = 35 #MMscfd
k = 1.3

## 2a Reciprocating Compressor

In [97]:
#Find Z

Ppc = 709.604 - 58.718*SG
Tpc = 170.491 + 307.344*SG

Tpr = Ts/Tpc
Ppr = Ps/Ppc

Tpr,Ppr

(1.320952565313899, 0.3772846851393297)

In [98]:
def z(Tpr,Ppr):
    Az = 1.39*(Tpr-0.92)**0.5-0.36*Tpr-0.1
    Cz = 0.132 - 0.32*np.log10(Tpr)
    Ez = 9*(Tpr-1)
    Fz = 0.3106 - 0.49*Tpr+0.1824*Tpr**2
    Dz = 10**Fz
    Bz = (0.62-0.23*Tpr)*Ppr +(0.066/(Tpr-0.86)-0.037)*Ppr**2+0.32*Ppr**6/10**Ez

    zCorr = Az + (1-Az)/np.exp(Bz) + Cz * Ppr**Dz
    return zCorr

In [99]:
za = z(Tpr,Ppr)

In [100]:
za

0.9492043323965326

In [101]:
# -w analytical approach for reciprocating compressor (hp/MMscfd)

a1 = (k/(k-1))*3.027*(Pb/Tb)*Ts*((Pd/Ps)**(za*(k-1)/k)-1)
a1

43.11186578471977

In [102]:
#standar q to cfm (as q1) calculation

q0 = q*1e6/24/60
q1 = (14.7/250)*(550/520)*q0
q0,q1

(24305.555555555555, 1511.6185897435896)

In [103]:
#Polytropic Effeciency

Ep = 0.61+0.03*np.log10(q1)
Ep

0.7053832667257183

In [104]:
#bhp/MMscfd

a1/Ep

61.11835624459662

In [105]:
#total bhp

a1/Ep*35

2139.1424685608818

## 2b Centrifugal Compressor

In [106]:
#Polytropic Ratio Calculation
Rp = (k-1)/k/Ep
Rp

0.3271543877705328

In [107]:
#Discharge Temperature Calculation
Td = Ts*(Pd/Ps)**(Rp)
Td

732.4035918503201

In [108]:
#T discharge in Fahreinheit
Td-460

272.40359185032014

In [110]:
(zs/zd)*(Pb/Ps)*(Ts/Tb)*q0

1501.4216840551285

In [170]:
#Z Factor Calculation for Discharge and Suction condition
#S: suction, 250 psi and 90 deg F (550 R)
#D: discharge, 600 psi and 272 deg F (732 R)

zs = z(Ts/Tpc,Ps/Ppc)
zd = z(Td/Tpc,Pd/Ppc)

zs,zd

(0.9492043323965326, 0.9556508538230679)

In [186]:
#Convergency Test

eps = 1e-2
l = 0
zd0 = zd
for i in range(0,1000):
    q1_i = (zs/zd0)*(Pb/Ps)*(Ts/Tb)*q0
    Ep_i = 0.61 + 0.03 * np.log10(q1_i)
    Rp_i = (k-1)/k/Ep_i
    Td_i = Ts*(Pd/Ps)**Rp_i

    zd0 = z(Td_i/Tpc,Pd/Ppc)
    l = l+1
zs,zd0

#Result: It is not going to be convergence

(0.9492043323965326, 0.9556576872465696)

In [189]:
HPg = ((q1_i*Ps)/(229*Ep_i))*((zs+zd0)/(2*zs))*(((Pd/Ps)**Rp_i-1)/Rp_i)
HPg

2363.9224525149752

In [ ]:
HPg_total = HPg + 50